<a href="https://colab.research.google.com/github/jihyungkim94/CAP-5610/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1 - K-Means Clustering with Real World Dataset

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import random 
from scipy.spatial import distance
from sklearn import datasets
from sklearn.metrics import accuracy_score
from collections import Counter

Define several auxiliary functions used for Task 1.

In [ ]:
# Function to calculate the SSE given some distance function
def calculate_sse(clusters, centers, dist_func):
    sse = 0 
    for i, center in enumerate(centers):
        # get the cluster associated with this center
        cluster = clusters[i]
        
        # sum up the squared errors for all data points in this cluster and add to the running sse
        cluster_error = 0
        for x in cluster:
            cluster_error += dist_func(x, center) ** 2

        sse += cluster_error
    
    return sse

# Generalized Jaccard distance function
def generalized_jaccard(a, b):
    num = 0
    denom = 0
    for i in range(np.shape(a)[0]):
        num += min(a[i], b[i])
        denom += max(a[i], b[i])
    
    coeff = 0 if denom == 0 else num/denom
        
    return 1-coeff

In [ ]:
iris = datasets.load_iris()
X = iris.data
Y = iris.target

# labels = [0,1,2]
k = len(np.unique(Y))

Randomly choose 3 initial centers.

In [ ]:
np.random.seed(0)
random_indices = np.random.choice(X.shape[0], size=3, replace=False)
initial_centers = X[random_indices]
print('Initial centers (randomly chosen):\n', initial_centers)

Initial centers (randomly chosen):
 [[5.8 2.8 5.1 2.4]
 [6.  2.2 4.  1. ]
 [5.5 4.2 1.4 0.2]]


Q1) K-means using Euclidean distance

In [ ]:
clf = KMeans(
    criterion='sse',
    dist_func=distance.euclidean, 
    initial_centroids=initial_centers,
    k=k,
)

centers, clusters, cluster_assign = clf.fit(X)
y_predict = np.zeros(len(Y), dtype=int)

centers = np.array(centers)
cluster_assign = np.array(cluster_assign)
for i in range(k):
    cluster_indices = np.where(cluster_assign == i)
    label = Counter(Y[cluster_indices]).most_common(1)[0][0]
    y_predict[cluster_indices] = label
    
print('SSE: ', calculate_sse(clusters, centers, dist_func=distance.euclidean))
print('Accuracy: ', accuracy_score(Y, y_predict))
print('Num iterations: ', clf.num_iterations)

SSE:  78.8556658259773
Accuracy:  0.8866666666666667
Num iterations:  9


Q2) K-means using cosine

In [ ]:
clf = KMeans(
    criterion='centroid',
    dist_func=distance.cosine, 
    initial_centroids=initial_centers,
    k=k,
)

centers, clusters, cluster_assign = clf.fit(X)
y_predict = np.zeros(len(Y), dtype=int)

centers = np.array(centers)
cluster_assign = np.array(cluster_assign)
for i in range(k):
    cluster_indices = np.where(cluster_assign == i)
    label = Counter(Y[cluster_indices]).most_common(1)[0][0]
    y_predict[cluster_indices] = label
    
print('SSE: ', calculate_sse(clusters, centers, dist_func=distance.cosine))
print('Accuracy: ', accuracy_score(Y, y_predict))
print('Num iterations: ', clf.num_iterations)

SSE:  0.00034526683520035636
Accuracy:  0.9666666666666667
Num iterations:  4


Q3) K-means using Jaccard similarity

In [ ]:
clf = KMeans(
    criterion='centroid',
    dist_func=generalized_jaccard, 
    initial_centroids=initial_centers,
    k=k,
)

centers, clusters, cluster_assign = clf.fit(X)
y_predict = np.zeros(len(Y), dtype=int)

centers = np.array(centers)
cluster_assign = np.array(cluster_assign)
for i in range(k):
    cluster_indices = np.where(cluster_assign == i)
    prediction = Counter(Y[cluster_indices]).most_common(1)
    label = prediction[0][0] if prediction else 0
    y_predict[cluster_indices] = label

print('SSE: ', calculate_sse(clusters, centers, dist_func=generalized_jaccard))
print('Accuracy: ', accuracy_score(Y, y_predict))
print('Num iterations: ', clf.num_iterations)

SSE:  1.0542011755590268
Accuracy:  0.8933333333333333
Num iterations:  4


Q4 - Number of iterations with different terminating conditions

In [ ]:
# if criterion is None we run through max_iter=100 iterations of k-means
criteria_names = ['centroid', 'SSE', 'max preset value']
criteria = ['centroid', 'sse', None]

dist_names = ['Euclidean distance', 'Cosine', 'Generalized Jaccard']
dist_funcs = [distance.euclidean, distance.cosine, generalized_jaccard]

for dist_func, dist_name in zip(dist_funcs, dist_names):
    print('K-means using {}:'.format(dist_name))
    for criterion, criterion_name in zip(criteria, criteria_names):
            clf = KMeans(
            criterion=criterion,
            dist_func=dist_func, 
            initial_centroids=initial_centers,
            k=k,
            max_iter=100
        )
            clf.fit(X)
            print('Num iterations using {} as stop criterion: {}'.format(criterion_name, clf.num_iterations))
    print('\n')

K-means using Euclidean distance:
Num iterations using centroid as stop criterion: 9
Num iterations using SSE as stop criterion: 9
Num iterations using max preset value as stop criterion: 100


K-means using Cosine:
Num iterations using centroid as stop criterion: 4
Num iterations using SSE as stop criterion: 4
Num iterations using max preset value as stop criterion: 100


K-means using Generalized Jaccard:
Num iterations using centroid as stop criterion: 4
Num iterations using SSE as stop criterion: 4
Num iterations using max preset value as stop criterion: 100




# Task 2

Data for clusters A and B

In [ ]:
A = np.array([
    [4.7, 3.2],
    [4.9, 3.1],
    [5.0, 3.0],
    [4.6, 2.9]
])

B = np.array([
    [5.9, 3.2],
    [6.7, 3.1],
    [6.0, 3.0],
    [6.2, 2.8]
])

Compute a distance matrix where distance_matrix[i][j] represents the distance between A[i] and B[j]

In [ ]:
distance_matrix = np.zeros((4,4))

for i in range(A.shape[0]):
    for j in range(B.shape[0]):
        distance_matrix[i][j] = distance.euclidean(A[i], B[j])
        
distance_matrix

array([[1.2       , 2.00249844, 1.31529464, 1.55241747],
       [1.00498756, 1.8       , 1.1045361 , 1.33416641],
       [0.92195445, 1.70293864, 1.        , 1.21655251],
       [1.33416641, 2.10950231, 1.40356688, 1.60312195]])

A. Distance between two farthest members

In [ ]:
max_indices = np.unravel_index(distance_matrix.argmax(), distance_matrix.shape)

print('Max distance: ', round(distance_matrix[max_indices], 4))
print('A: {}, B: {}'.format(A[max_indices[0]], B[max_indices[1]]))

Max distance:  2.1095
A: [4.6 2.9], B: [6.7 3.1]


B. Distance between two closest members

In [ ]:
min_indices = np.unravel_index(distance_matrix.argmin(), distance_matrix.shape)

print('Min distance: ', round(distance_matrix[min_indices], 4))
print('A: {}, B: {}'.format(A[min_indices[0]], B[min_indices[1]]))

Min distance:  0.922
A: [5. 3.], B: [5.9 3.2]


C. Average distance between all pairs

In [ ]:
print('Average distance between cluster pairs: ', round(np.mean(distance_matrix), 4))

Average distance between cluster pairs:  1.4129
